# NLP Project ENSAE 2023

# <span style="color:orange"> Text Similarity Techniques for Services Categorization and Search </span>

In [4]:
import numpy as np
import pandas as pd
df = pd.read_csv('df').drop('Unnamed: 0', axis=1)

In [5]:
df.head()

,Nom et prénom,Numéro de téléphone,Service proposé,lowerplot
0,Capucine Remy,0643621859,Gestion de projet de construction,gestion projet construction
1,Théophile Dias-Coste,0610883745,Développement d'applications mobiles,développement dapplications_mobile
2,Valentine Denis-Briand,0643864969,Rédaction de contenu marketing,rédaction contenu marketing
3,Marc de Charrier,0682454911,Traduction de documents juridiques,traduction_document juridiques
4,Daniel Maillard-Ramos,0636373700,Analyse de données pour les entreprises,analyse données entreprises


## <span style="color:orange"> TF-IDF *(textual vectorization method)* based Search Engine </span>

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.translate.bleu_score import sentence_bleu
from termcolor import colored
from tabulate import tabulate

In [12]:
import nltk
from nltk.tokenize import TreebankWordTokenizer, TweetTokenizer
from gensim.models.phrases import Phrases, Phraser
#nltk.download('punkt')
#sent_detector = nltk.data.load('tokenizers/punkt/english.pickle')

**TfidfVectorizer** is a scikit-learn class that allows to transform a text corpus into a TF-IDF vector matrix. 

In [9]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df['lowerplot'])

### 1. Cosine Similarity

In [91]:
def search_engine():
    print(colored("Bonjour et bienvenue sur notre moteur de recherche local de Bastia !\n \nEntrez des mots-clés pour être mis en relation avec les personnes susceptibles de vous \nrendre service près de chez vous.","magenta"))
    query = input(colored('\nMot clé : ','blue'))
    while query.lower() != "non" :
        query_tokens = query.split() # convertir l'input en une liste de mots
        
        query_vector = vectorizer.transform([query])
        results      = cosine_similarity(X, query_vector)
        
        relevant_candidates = np.where(results.flatten() > 0.2)[0] # trouver seuil pertinent
        sorted_candidates = np.argsort(results.flatten()[relevant_candidates])[::-1]
        all_candidates = relevant_candidates[sorted_candidates]

        print('\033[45m\033[37m\nRésultats de la recherche pour "{}": \033[0m\n'.format(query))
        headers = ['Nom', 'Contact', 'Service']#, 'Score BLEU']
        rows = []
        for i in all_candidates:
            name    = df.iloc[i, 0]
            number  = df.iloc[i, 1]
            service = df.iloc[i, 2]
            serv    = df.iloc[i, 3]
            
            candidate = serv
            reference = [query]
            score = round(nltk.translate.bleu_score.sentence_bleu(reference, candidate)*100,2) # calculer le score BLEU
            rows.append([name, number, service])#, score])

        print(tabulate(rows, headers=headers, tablefmt='orgtbl'))

        query = input(colored('\nAvez-vous besoin d\'autre chose ?\n','blue'))
    print('\033[45m\033[37m\nÀ bientôt et merci d\'avoir utilisé le moteur de recherche local ! \033[0m')
    return()

In [92]:
search_engine()

Bonjour et bienvenue sur notre moteur de recherche local de Bastia !
 
Entrez des mots-clés pour être mis en relation avec les personnes susceptibles de vous 
rendre service près de chez vous.

Mot clé : nlp


/Users/louisacamadini/anaconda3/lib/python3.7/site-packages/nltk/translate/bleu_score.py:523: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)



Résultats de la recherche pour "nlp": 

| Nom             | Contact    | Service                      |
|-----------------+------------+------------------------------|
| Pierre Colombo  | 06XXXXXXXX | Professeur de NLP            |
| Louisa Camadini | 0646674880 | Effectuer vos projets de NLP |

Avez-vous besoin d'autre chose ?
Danse

Résultats de la recherche pour "Danse": 

| Nom                      |    Contact | Service                                                                       |
|--------------------------+------------+-------------------------------------------------------------------------------|
| Étienne-Thomas Delorme   | 0649670799 | Danse classique                                                               |
| Sébastien De Sousa-Lebon | 0610611931 | Danse jazz                                                                    |
| Danielle Fleury          | 0607632945 | Professeur de danse à domicile                                                |
| Jules Du

()

### 2. Euclidian distance

In [38]:
def search_engine_eucl():
    print(colored("Bonjour et bienvenue sur notre moteur de recherche local de Bastia !\n \nEntrez des mots-clés pour être mis en relation avec les personnes susceptibles de vous \nrendre service près de chez vous.","magenta"))
    query = input(colored('\nMot clé : ','blue'))
    while query.lower() != "non" :
        query_vector = vectorizer.transform([query]).toarray()
        query_vector = np.reshape(query_vector, (query_vector.shape[1],))
        results      = np.linalg.norm(X - query_vector, axis=1)
        
        relevant_candidates = np.where(results < 1)[0] # trouver seuil pertinent
        sorted_candidates = np.argsort(results[relevant_candidates])
        all_candidates = relevant_candidates[sorted_candidates]
        
        print('\033[45m\033[37m\nRésultats de la recherche pour "{}": \033[0m\n'.format(query))
        headers = ['Nom', 'Contact', 'Service']#, 'Score BLEU']
        rows = []
        for i in all_candidates:
            name    = df.iloc[i, 0]
            number  = df.iloc[i, 1]
            service = df.iloc[i, 2]
            serv    = df.iloc[i, 3]
            
            candidate = serv
            reference = [query]
            score = round(nltk.translate.bleu_score.sentence_bleu(reference, candidate)*100,2) # calculer le score BLEU
            rows.append([name, number, service])#, score])

        print(tabulate(rows, headers=headers, tablefmt='orgtbl'))

        query = input(colored('\nAvez-vous besoin d\'autre chose ?\n','blue'))
    print('\033[45m\033[37m\nÀ bientôt et merci d\'avoir utilisé le moteur de recherche local ! \033[0m')
    return()

In [39]:
search_engine_eucl()

Bonjour et bienvenue sur notre moteur de recherche local de Bastia !
 
Entrez des mots-clés pour être mis en relation avec les personnes susceptibles de vous 
rendre service près de chez vous.

Mot clé : NLP

Résultats de la recherche pour "NLP": 

| Nom             | Contact    | Service                      |
|-----------------+------------+------------------------------|
| Pierre Colombo  | 06XXXXXXXX | Professeur de NLP            |
| Louisa Camadini | 0646674880 | Effectuer vos projets de NLP |

Avez-vous besoin d'autre chose ?
Danse

Résultats de la recherche pour "Danse": 

| Nom                      |    Contact | Service                          |
|--------------------------+------------+----------------------------------|
| Étienne-Thomas Delorme   | 0649670799 | Danse classique                  |
| Sébastien De Sousa-Lebon | 0610611931 | Danse jazz                       |
| Danielle Fleury          | 0607632945 | Professeur de danse à domicile   |
| Jules Dufour             

()

### 3. Pearson correlation 

In [49]:
from scipy.spatial.distance import pdist, squareform
def search_engine_pearson():
    print(colored("Bonjour et bienvenue sur notre moteur de recherche local de Bastia !\n \nEntrez des mots-clés pour être mis en relation avec les personnes susceptibles de vous \nrendre service près de chez vous.","magenta"))
    query = input(colored('\nMot clé : ','blue'))
    while query.lower() != "non" :
        query_vector = vectorizer.transform([query])
        pairwise_distances = pdist(X.toarray(), metric='correlation')
        pairwise_similarities = 1 - squareform(pairwise_distances)
        query_similarities = 1 - pdist(np.vstack((X.toarray(), query_vector.toarray())), metric='correlation')
        results = query_similarities.flatten()[:-1]
        
        relevant_candidates = np.where(results > 0.5)[0] # trouver seuil pertinent
        sorted_candidates = np.argsort(results[relevant_candidates])[::-1]
        all_candidates = relevant_candidates[sorted_candidates]
        
        print('\033[45m\033[37m\nRésultats de la recherche pour "{}": \033[0m\n'.format(query))
        headers = ['Nom', 'Contact', 'Service']#, 'Score BLEU']
        rows = []
        for i in all_candidates:
            if i <= len(df) :
                name    = df.iloc[i, 0]
                number  = df.iloc[i, 1]
                service = df.iloc[i, 2]
                rows.append([name, number, service])
        print(tabulate(rows, headers=headers, tablefmt='orgtbl'))
        query = input(colored('\nAvez-vous besoin d\'autre chose ?\n','blue'))
    print('\033[45m\033[37m\nÀ bientôt et merci d\'avoir utilisé le moteur de recherche local ! \033[0m')
    return()

In [50]:
search_engine_pearson()

Bonjour et bienvenue sur notre moteur de recherche local de Bastia !
 
Entrez des mots-clés pour être mis en relation avec les personnes susceptibles de vous 
rendre service près de chez vous.

Mot clé : NLP

Résultats de la recherche pour "NLP": 

| Nom           |    Contact | Service             |
|---------------+------------+---------------------|
| Adèle du Ruiz | 0663238554 | Cours de leadership |

Avez-vous besoin d'autre chose ?
non

À bientôt et merci d'avoir utilisé le moteur de recherche local ! 


()

## <span style="color:orange"> BERT Embedding Method </span>

In [56]:
import transformers
import sentence_transformers

In [1]:
from sentence_transformers import SentenceTransformer

In [89]:
model = SentenceTransformer('distilbert-base-nli-mean-tokens')
X_bert = model.encode(df['lowerplot'])

def search_engine_bert():
    print(colored("Bonjour et bienvenue sur notre moteur de recherche local de Bastia !\n \nEntrez des mots-clés pour être mis en relation avec les personnes susceptibles de vous \nrendre service près de chez vous.","magenta"))
    query = input(colored('\nMot clé : ','blue'))
    while query.lower() != "non" :
        query_vector = model.encode([query])
        results      = cosine_similarity(X_bert, query_vector)

        relevant_candidates = np.where(results.flatten() > 0.9)[0] # trouver seuil pertinent
        sorted_candidates = np.argsort(results.flatten()[relevant_candidates])[::-1]
        all_candidates = relevant_candidates[sorted_candidates]

        print('\033[45m\033[37m\nRésultats de la recherche pour "{}": \033[0m\n'.format(query))
        headers = ['Nom', 'Contact', 'Service']#, 'Score BLEU']
        rows = []
        for i in all_candidates:
            print(i)
            name    = df.iloc[i, 0]
            number  = df.iloc[i, 1]
            service = df.iloc[i, 2]
            serv    = df.iloc[i, 3]
            
            candidate = model.encode([serv])
            reference = model.encode([query])
            score = round(cosine_similarity(reference, candidate)[0][0]*100, 2)
            rows.append([name, number, service])

        print(tabulate(rows, headers=headers, tablefmt='orgtbl'))

        query = input(colored('\nAvez-vous besoin d\'autre chose ?\n','blue'))
    print('\033[45m\033[37m\nÀ bientôt et merci d\'avoir utilisé le moteur de recherche local ! \033[0m')
    return()


In [90]:
search_engine_bert()

Bonjour et bienvenue sur notre moteur de recherche local de Bastia !
 
Entrez des mots-clés pour être mis en relation avec les personnes susceptibles de vous 
rendre service près de chez vous.

Mot clé : nlp

Résultats de la recherche pour "nlp": 

1126
| Nom            | Contact    | Service           |
|----------------+------------+-------------------|
| Pierre Colombo | 06XXXXXXXX | Professeur de NLP |

Avez-vous besoin d'autre chose ?
danse

Résultats de la recherche pour "danse": 

423
404
406
| Nom                    |    Contact | Service             |
|------------------------+------------+---------------------|
| Cécile Le Baron        | 0614905329 | Danse du ventre     |
| Étienne-Thomas Delorme | 0649670799 | Danse classique     |
| Jules Berthelot        | 0679258757 | Danse contemporaine |

Avez-vous besoin d'autre chose ?
non

À bientôt et merci d'avoir utilisé le moteur de recherche local ! 


()

## <span style="color:orange"> Word2Vec Embedding Method </span>

In [69]:
from gensim.models import Word2Vec

# train a Word2Vec model on our text corpus
sentences = [nltk.tokenize.word_tokenize(text.lower()) for text in df['lowerplot']]
model = Word2Vec(sentences, size=100, window=5, min_count=1, workers=4)

def embed_query(query_tokens, model):
    # embed the query as the average of its constituent word vectors
    query_vector = np.mean([model.wv[token] for token in query_tokens if token in model.wv], axis=0)
    return query_vector

def search_engine_w2v():
    print(colored("Bonjour et bienvenue sur notre moteur de recherche local de Bastia !\n \nEntrez des mots-clés pour être mis en relation avec les personnes susceptibles de vous \nrendre service près de chez vous.","magenta"))
    query = input(colored('\nMot clé : ','blue'))
    while query.lower() != "non" :
        query_tokens = nltk.tokenize.word_tokenize(query.lower())
        
        query_vector = embed_query(query_tokens, model)
        results      = np.dot(model.wv.vectors, query_vector)
        
        relevant_candidates = np.where(results > 0.1)[0] # trouver seuil pertinent
        sorted_candidates = np.argsort(results[relevant_candidates])[::-1]
        all_candidates = relevant_candidates[sorted_candidates]

        print('\033[45m\033[37m\nRésultats de la recherche pour "{}": \033[0m\n'.format(query))
        headers = ['Nom', 'Contact', 'Service']#, 'Score BLEU']
        rows = []
        for i in all_candidates:
            if i <= len(df) :
                name    = df.iloc[i, 0]
                number  = df.iloc[i, 1]
                service = df.iloc[i, 2]
                serv    = df.iloc[i, 3]
            
                candidate = serv
                reference = [query]
                score = round(nltk.translate.bleu_score.sentence_bleu(reference, candidate)*100,2) # calculer le score BLEU
                rows.append([name, number, service])#, score])

        print(tabulate(rows, headers=headers, tablefmt='orgtbl'))

        query = input(colored('\nAvez-vous besoin d\'autre chose ?\n','blue'))
    print('\033[45m\033[37m\nÀ bientôt et merci d\'avoir utilisé le moteur de recherche local ! \033[0m')
    return()


In [70]:
search_engine_w2v()

Bonjour et bienvenue sur notre moteur de recherche local de Bastia !
 
Entrez des mots-clés pour être mis en relation avec les personnes susceptibles de vous 
rendre service près de chez vous.

Mot clé : NLP

Résultats de la recherche pour "NLP": 

| Nom   | Contact   | Service   |
|-------+-----------+-----------|

Avez-vous besoin d'autre chose ?
danse

Résultats de la recherche pour "danse": 

| Nom   | Contact   | Service   |
|-------+-----------+-----------|

Avez-vous besoin d'autre chose ?
non

À bientôt et merci d'avoir utilisé le moteur de recherche local ! 


()